# TP 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.dates as md
import seaborn as sns
import datetime as datetime
import numpy as np
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
Limpieza_Clicks = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Clicks.ipynb'))
Limpieza_Auctions = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Auctions.ipynb'))
Limpieza_Installs = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Installs.ipynb'))
Limpieza_Events = '"{}"'.format(os.path.join(current_folder, '..', 'Notebook_limpieza', 'Limpieza_Events.ipynb'))

In [20]:
%run $Limpieza_Clicks
%run $Limpieza_Auctions
%run $Limpieza_Installs
%run $Limpieza_Events

## <u>St(d):</u> Estimar el tiempo hasta que el dispositivo vuelva a aparecer en subastas.

In [14]:
auctions['count'] = auctions.groupby('device_id')['date'].transform('count')

In [15]:
df = auctions.loc[auctions['count']>1] #Se eliminan los device que no volvieron a aparecer en subastas.
df.sort_values(by='date',ascending=True,inplace=True)

### La siguiente funcion con groupby demora aproximadamente 30mins..  MEJORAR!!!!!!!

In [19]:
def fun_aux(serie):   
    s = serie.reset_index(drop=True)
    seconds = 0
    if len(s)==1:
        seconds = (s[1]-s[0]).total_seconds()
    for i in range(1,len(s)):
        seconds += ((s[i]-s[i-1]).total_seconds())
    return (seconds/len(s))

df['st'] = df.groupby('device_id')['date'].transform(fun_aux)

In [21]:
df[['device_id','st']].sort_values(by='st',ascending=False).drop_duplicates()

,device_id,st
8086222,7853085480790637208,385967.394140
13675114,1269725033831458756,385649.953807
2634198,7860813783813334575,385456.956811
13166628,2475616601707472135,384223.127566
11165166,1142906158263162794,382415.345068
1142321,3269561262669350087,382271.382494
13166950,2099776725702693163,381993.734187
9137348,6116191750169987062,381763.453320
14768553,2707895775047985180,381221.001334
9322772,2472612324130300502,380803.543033


# Ejemplo KNN

Previo a esto hay que generar algunos features y tener el set de labels para realizar la separacion de train y test

In [ ]:
set_training = labels.merge(df_features, how='inner', on='device_id')

In [ ]:
columns = list(set_training.columns)
columns.remove('label')

In [ ]:
from sklearn.model_selection import train_test_split

#TRAIN_TEST_SPLIT
#Split arrays or matrices into random train and test subsets
X_train, X_test, Y_train, Y_test = train_test_split(set_training.loc[:, columns],
                                                    set_training['label'],
                                                    test_size=0.2,
                                                    random_state=43)

Lo siguiente se lo podria realizan n veces para probar distintos valors de k ante una misma division de set de training

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

k=5 #-->Hiperparametro
neigh = KNeighborsRegressor(n_neighbors = k ,    #Cantidad de vecinos a usar --> K hiperparametro.
                            n_jobs=-1,           #Define la cantidad de procesadores a usar, -1 todos los disponibles.
                            weights='distance',  #Se define el peso como la inversa a la distancia, los mas cercanos tienen mayor influencia.
                            algorithm='brute'    #Fuerza bruta para el algorimo, podría ser KDTree o sino automatico.
                           )

#En weights se puede pasar una funcion definida por el usuario--> Segundo hiperparametro posible de KNN.

neigh.fit(X_train,Y_train) #Fit the model using X as training data and Y as target values
predicted = neigh.predict(X_test) #Predict the target for the provided data
print(k,"Prediccion: ",metrics.roc_auc_score(Y_test, predicted))
#--> Aca deberia imprimir el score del algoritmo para un determinado valor de k

#ROC_AUC_SCORE
#Compute Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.